<h1><center>Data Scraping</center></h1>
This notebook contains an example of scraping 2 cv's. The extraction of all the documents takes roughly a full day,
and can be done using the script called ExtractLattes.py, located in the folder data_scraping.

This file uses the script BreakingCaptcha.py, present also in data_scraping, which has the function implemented
to try to guess the captcha. The file "sinais.pkl" contains information about the captcha that is used
by the script BrakingCaptcha.py in order to make the guess.

In [1]:
import sys
sys.path.insert(1, './data_scraping')

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from BreakingCaptcha import *
from PIL import Image
from io import BytesIO
import base64
import os
import logging

logging.basicConfig(filename='./data_scraping/extract_lattes.log',level=logging.INFO)

def Get_Captcha(ele_captcha,outputname):
    # get the captcha as a base64 string
    img_captcha_base64 = driver.execute_async_script("""
        var ele = arguments[0], callback = arguments[1];
        ele.addEventListener('load', function fn(){
          ele.removeEventListener('load', fn, false);
          var cnv = document.createElement('canvas');
          cnv.width = this.width; cnv.height = this.height;
          cnv.getContext('2d').drawImage(this, 0, 0);
          callback(cnv.toDataURL('image/jpeg').substring(22));
        }, false);
        ele.dispatchEvent(new Event('load'));
        """, ele_captcha)

    # save the captcha to a file
    with open(outputname, 'wb') as f:
        f.write(base64.b64decode(img_captcha_base64))



Above we define a function to extract the image of the captcha from the website. Notice that one must
use Firefox for this. We cannot say that it will work with other browsers.

Below we actually extract two cvs.

In [2]:
options = Options()
# options.headless = True

donwload_file_location = '/home/davi/Dropbox/FGV/EMAp/pbmnet/data_scraping/cvs/'
profile = webdriver.FirefoxProfile()
profile.set_preference('browser.download.folderList', 2) # custom location
profile.set_preference('browser.download.manager.showWhenStarting', False)
profile.set_preference('browser.download.dir', donwload_file_location)
profile.set_preference('browser.helperApps.neverAsk.saveToDisk', 'application/zip')

driver = webdriver.Firefox(profile,options=options)
driver.set_script_timeout(10)
root_page   =  "http://buscatextual.cnpq.br/buscatextual/download.do?metodo=apresentar&idcnpq="

lista_cv = ['6778809047429890','0289400810360557']

for cod_cv in lista_cv:
    url = root_page + cod_cv
    driver.get(url)
    for i in range(0,10):
        ele_captcha = driver.find_element_by_xpath("//img[contains(./@src, 'getImagemCaptcha')]")
        outputname = './data_scraping/captcha/captcha_'+cod_cv+'.jpg'
        Get_Captcha(ele_captcha,outputname)
        previsao = BreakCaptcha(outputname)
        inputElement = driver.find_element_by_id("informado")
        inputElement.send_keys(previsao)
        driver.find_element_by_id('btn_validar_captcha').click()
        check_download = os.path.isfile('./data_scraping/cvs/{}.zip'.format(cod_cv))
        print(check_download)
        if check_download == True:
            logging.info('{} - SUCESSO'.format(cod_cv))
            break
        if i == 9:
            logging.warning("{} - FALHOU".format(cod_cv))
driver.quit()

True
True
